# Trabalho Machine Learning I - Santander Coders Data Science 2023

Integrantes:
- Bruno Henrique
- Fernanda Beatriz Dallaqua
- Natália Alves Brito
- Vinícius D' Lucas

In [1]:
import pandas as pd

In [3]:
path_train = 'playground-series-s3e2/train.csv'
df         = pd.read_csv(path_train)
df.head(10)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0
1,1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0
2,2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,0
3,3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0
4,4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0
5,5,Female,34.0,0,0,Yes,Private,Urban,84.35,22.2,Unknown,0
6,6,Female,53.0,0,0,Yes,Private,Rural,88.97,25.3,never smoked,0
7,7,Male,78.0,0,1,Yes,Self-employed,Rural,75.32,24.8,Unknown,0
8,8,Female,45.0,0,0,Yes,Private,Rural,107.22,34.1,never smoked,0
9,9,Female,62.0,0,0,Yes,Govt_job,Urban,62.68,18.4,formerly smoked,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15304 entries, 0 to 15303
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 15304 non-null  int64  
 1   gender             15304 non-null  object 
 2   age                15304 non-null  float64
 3   hypertension       15304 non-null  int64  
 4   heart_disease      15304 non-null  int64  
 5   ever_married       15304 non-null  object 
 6   work_type          15304 non-null  object 
 7   Residence_type     15304 non-null  object 
 8   avg_glucose_level  15304 non-null  float64
 9   bmi                15304 non-null  float64
 10  smoking_status     15304 non-null  object 
 11  stroke             15304 non-null  int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 1.4+ MB


In [12]:
def get_valores_unicos(df, coluna):
    print(f'Os valores únicos para a coluna {coluna} são:')
    print(list(df[coluna].unique()))
    print()

In [13]:
get_valores_unicos(df,'gender')
get_valores_unicos(df,'ever_married')
get_valores_unicos(df,'work_type')
get_valores_unicos(df,'Residence_type')
get_valores_unicos(df,'smoking_status')
get_valores_unicos(df,'stroke')

Os valores únicos para a coluna gender são:
['Male', 'Female', 'Other']

Os valores únicos para a coluna ever_married são:
['Yes', 'No']

Os valores únicos para a coluna work_type são:
['Private', 'Self-employed', 'Govt_job', 'children', 'Never_worked']

Os valores únicos para a coluna Residence_type são:
['Urban', 'Rural']

Os valores únicos para a coluna smoking_status são:
['never smoked', 'formerly smoked', 'Unknown', 'smokes']

Os valores únicos para a coluna stroke são:
[0, 1]



# Considerações

Transformar coluna gender em 3. Uma para Male, outra para Female e outra para Other.
Mesma lógica para work_type e residence_type

Para a coluna smoking_status talvez o melhor seja dar os níveis de intensidade mesmo (0 para never smoked, 1 para formely smoked e 2 para smokes) e talvez retirar o Unknown e colocar em uma nova coluna 